# 要約 
このJupyter Notebookでは、Kaggleの「LLM 20 Questions」コンペティションに参加するためのエージェントを作成するプロセスが示されています。エージェントは、「20の質問」ゲームをプレイするために設計されており、ユーザーの質問に対して「はい」または「いいえ」で答える役割を持つ「回答者LLM」と、その質問を行い、ターゲットを推測する役割の「質問者LLM」で構成されています。最終的に、実行すると提出用の`submission.tar.gz`ファイルが生成されます。

### 問題解決のアプローチ
1. **環境設定とライブラリのインストール**:
   - `immutabledict`や`sentencepiece`といったライブラリをインストールし、Googleの`gemma_pytorch`リポジトリからモデルの一部を克隆します。

2. **エージェントの定義**:
   - **GemmaFormatter**クラスが実装されており、プロンプトの構成を担当します。また、質問者と回答者のターンを管理するためのメソッドを提供します。
   - **GemmaAgent**を基にした`GemmaQuestionerAgent`および`GemmaAnswererAgent`クラスが実装され、質問者と回答者の具体的な動作を定義します。

3. **モデルの初期化**:
   - `GemmaForCausalLM`を使用してモデルが初期化され、与えられた重みを読み込みます。モデルはCUDAデバイス上で動作するように設定されています。

4. **セッション管理**:
   - 各エージェントはセッションを開始し、ユーザーからの観察（`obs`）データに基づいて適切なプロンプトを生成します。これにより、質問の生成やキーワードの推測が行われます。

5. **エージェントの呼び出し**:
   - `agent_fn`関数がエージェントを選択し、観察データに基づいてそれぞれのエージェント（質問者または回答者）を呼び出します。

### 使用されているライブラリ
- **torch**: PyTorchライブラリで深層学習モデルの構築と学習を支援します。
- **gemma.pytorch**: モデルの設定と実行を支援するために使用されるGoogleのリポジトリからのライブラリです。

このNotebookは、20の質問ゲームを効率良くプレイするためのLLMのエージェントを構築するためのフレームワークを提供し、ユーザーの質問に対する適切な応答を生成する方法を示しています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に関連する、機械学習・深層学習において初心者がつまずきそうなマイナーな専門用語の解説を示します。

1. **GemmaFormatter**:
   - 「フォーマッタ」は、入力データ（この場合はユーザーとモデルのターン）を特定の形式に変換するクラスです。言語モデルに与えるプロンプトの構造を定義し、質問や応答を体系的に管理します。

2. **GemmaForCausalLM**:
   - これは、因果的言語モデル（Causal Language Model）を実装するクラスであり、過去の単語に基づいて次の単語を予測するために設計されています。このノートブックでは、特定のタスク（20の質問ゲーム）に応じた応答を生成するために使用されます。

3. **サンプリング (sampling)**:
   - モデルが出力する可能性のある次の単語（トークン）を選ぶための手法のことです。このノートブックでは、`temperature`, `top_p`, `top_k` などのパラメータを使用してサンプリング戦略をチューニングします。これにより生成されるテキストの多様性を調整します。

4. **量子化 (quantization)**:
   - モデルの重みを圧縮してメモリの使用を削減する技術です。このノートブックでは、モデルが「量子化」されているかどうかを指定するために用いられます。特に、ディープラーニングモデルのデプロイ時に効率を向上させるために重要です。

5. **interleave_unequal関数**:
   - 2つのリストの要素を交互に組み合わせる関数です。しかし、リストの長さが異なる場合でも、両方のリストから要素を組み合わせて新しいリストを作成します。この機能は、質問と回答のペアリングに役立ちます。

6. **contextlib.contextmanager**:
   - Pythonのコンテキストマネージャを作成するためのユーティリティです。特定のリソース（この場合はデフォルトのテンソル型）を管理し、一時的な変更を行い後に元に戻すことを容易にします。これは、メモリ管理や状態管理の効率を向上させるのに役立ちます。

7. **reモジュール (re module)**:
   - 正規表現を使用するためのPythonの標準ライブラリの一部です。このノートブックでは、質問内容や回答内容を解析し、特定のパターンを見つけ出すために使用されています。

これらは、初心者にとってはあまり馴染みのない用語や概念である可能性が高く、実務を通じて初めて理解できるものが多いです。各用語がどういった役割を果たしているのか、またどのようにコード内で利用されているのかを抑えておくことが、理解を深める助けになります。

---


In [ ]:
# このノートブックは、LLM 20 Questions コンペティションにおけるエージェント作成プロセスを示しています。
# このノートブックを実行すると、直接提出できる submission.tar.gz ファイルが生成されます。

%%bash
# 作業環境の設定
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir -p /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

%%writefile submission/main.py
# エージェント用のメインスクリプト

import os
import sys
from pathlib import Path
import contextlib
import itertools
from typing import Iterable
import re

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# ノートブックとシミュレーション環境のためのシステムパスを設定
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

# 定数の定義
WEIGHTS_PATH = os.path.join(
    KAGGLE_AGENT_PATH if os.path.exists(KAGGLE_AGENT_PATH) else "/kaggle/input",
    "gemma/pytorch/7b-it-quant/2"
)

# プロンプト形式設定クラス
class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"  # ユーザーのターン
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"  # モデルのターン
        self.reset()  # 初期化

    def __repr__(self):
        return self._state  # 表示用

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)  # ユーザーのプロンプトを追加
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)  # モデルのプロンプトを追加
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"  # ユーザーのターンを開始
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"  # モデルのターンを開始
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"  # ターンを終了
        return self

    def reset(self):
        self._state = ""  # 状態のリセット
        if self._system_prompt:
            self.user(self._system_prompt)  # システムプロンプトを設定
        if self._few_shot_examples:
            self.apply_turns(self._few_shot_examples, start_agent='user')  # Few-shot例を適用
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)  # フォーマッタを循環させる
        for fmt, turn in zip(formatters, turns):
            fmt(turn)  # 各ターンを適用
        return self

# エージェントクラスの定義
@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """与えられたデータ型にデフォルトのtorchのdtypeを設定する。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)  # デフォルトデータ型をfloatに戻す

class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)  # デバイス設定
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルの初期化中")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()  # モデル設定を取得
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")  # トークナイザのパス
        model_config.quant = "quant" in variant  # 量子化の設定

        with _set_default_tensor_type(model_config.get_dtype()):  # デフォルトデータ型の設定
            self.model = GemmaForCausalLM(model_config)  # モデルのインスタンス化
            ckpt_path = os.path.join(WEIGHTS_PATH, f'gemma-{variant}.ckpt')  # チェックポイントのパス
            self.model.load_weights(ckpt_path)  # 重みを読み込む
            self.model.to(self._device).eval()  # モデルをデバイスに移動し評価モードに設定

    def __call__(self, obs, *args):
        self._start_session(obs)  # セッションを開始
        prompt = str(self.formatter)  # プロンプトを文字列として取得
        response = self._call_llm(prompt)  # LLMを呼び出す
        return self._parse_response(response, obs)  # レスポンスを解析して返す

    def _start_session(self, obs: dict):
        raise NotImplementedError  # 未実装のメソッド

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        sampler_kwargs = sampler_kwargs or {  # サンプリング設定
            'temperature': 0.01,
            'top_p': 0.1,
            'top_k': 1,
        }
        return self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)  # キーワードの解析
        return match.group().lower() if match else ''

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError  # 未実装のメソッド

def interleave_unequal(x, y):
    """2つのリストを交互に組み合わせるが、長さが異なる場合も考慮する。"""
    return [item for pair in itertools.zip_longest(x, y) for item in pair if item is not None]

class GemmaQuestionerAgent(GemmaAgent):
    def _start_session(self, obs):
        self.formatter.reset()  # フォーマッタのリセット
        self.formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を果たしています。")  # 初期プロンプトを設定
        turns = interleave_unequal(obs.questions, obs.answers)  # 質問と回答を交互に組み合わせ
        self.formatter.apply_turns(turns, start_agent='model')  # ターンを適用
        if obs.turnType == 'ask':
            self.formatter.user("はいまたはいいえで答えられる質問をしてください。")  # 質問の場合
        elif obs.turnType == 'guess':
            self.formatter.user("さて、キーワードを推測してください。推測は二重アスタリスクで囲んでください。")  # 推測の場合
        self.formatter.start_model_turn()  # モデルのターンを開始

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))  # 質問の解析
            return match.group() if match else "それは人ですか？"
        elif obs.turnType == 'guess':
            return self._parse_keyword(response)  # キーワードを解析して返す

class GemmaAnswererAgent(GemmaAgent):
    def _start_session(self, obs):
        self.formatter.reset()  # フォーマッタのリセット
        self.formatter.user(f"20の質問をプレイしましょう。あなたは回答者の役割を果たしています。キーワードは {obs.keyword} で、カテゴリは {obs.category} です。")  # 初期プロンプトを設定
        turns = interleave_unequal(obs.questions, obs.answers)  # 質問と回答を交互に組み合わせ
        self.formatter.apply_turns(turns, start_agent='user')  # ターンを適用
        self.formatter.user(f"この質問はキーワード {obs.keyword} に関するものです。はいまたはいいえで答えてください。答えは二重アスタリスクで囲んでください。")  # 回答のプロンプトを設定
        self.formatter.start_model_turn()  # モデルのターンを開始

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)  # キーワードを解析
        return 'yes' if 'yes' in answer else 'no'  # 答えを返す

# エージェントの作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者のはいまたはいいえでの質問に対して答えます。キーワードは特定の人、場所、または物です。"
few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を果たしています。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** さて、キーワードを推測してください。",
    "**フランス**", "正解です！",
]

# グローバルエージェント変数 - 並行して両方のエージェントを読み込まないようにする
agent = None

def get_agent(name: str):
    global agent

    if agent is None:  # エージェントが未初期化の場合
        if name == 'questioner':
            agent = GemmaQuestionerAgent(
                device='cuda:0',  # デバイス設定
                system_prompt=system_prompt,
                few_shot_examples=few_shot_examples,
            )
        elif name == 'answerer':
            agent = GemmaAnswererAgent(
                device='cuda:0',  # デバイス設定
                system_prompt=system_prompt,
                few_shot_examples=few_shot_examples,
            )
    assert agent is not None, "エージェントが初期化されていません。"  # エージェントの存在を確認
    return agent  # エージェントを返す

def agent_fn(obs, cfg):
    if obs.turnType in ["ask", "guess"]:
        response = get_agent('questioner')(obs)  # 質問者エージェントを呼び出す
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)  # 回答者エージェントを呼び出す
    return response if response and len(response) > 1 else "はい"  # レスポンスが有効であれば返す